# XShooter UVB Wavelengths

In [26]:
# imports
from pkg_resources import resource_filename
import os
import numpy as np
from astropy.io import fits
from astropy import units

from scipy.io import readsav

from pypeit.core.wave import airtovac
from pypeit.core.wavecal import waveio
from pypeit.core.wavecal import templates
from pypeit.core import wave
from pypeit.wavemodel import create_ThArlinelist

# New fussing

## Load up MagE wavelengths (and turn to air to ID with George's plots)

In [3]:
list_file = os.path.join(resource_filename('pypeit', 'data'), 'arc_lines', 'lists', 'ThAr_MagE_lines.dat')

In [4]:
mage_ThAr_vac = waveio.load_line_list(list_file)

In [6]:
mage_ThAr_vac[0:10]

wave,ion,NIST,Instr,amplitude,Source
float64,str4,int64,int64,float64,str12
3034.4515616388403,ThAr,1,32,3250.5330562821146,wavemodel.py
3094.2351186737014,ThAr,1,32,163.4091741852267,wavemodel.py
3140.0712478114897,ThAr,1,32,118.03246067794022,wavemodel.py
3170.5580546208257,ThAr,1,32,123.99931788498296,wavemodel.py
3181.14976954272,ThAr,1,32,160.1378083424093,wavemodel.py
3189.169932325525,ThAr,1,32,153.8608233826956,wavemodel.py
3196.585400263197,ThAr,1,32,104.46079002943156,wavemodel.py
3233.2380176392016,ThAr,1,32,150.16225080755922,wavemodel.py
3239.1470401332326,ThAr,1,32,130.3364122188992,wavemodel.py


In [11]:
mage_ThAr_air = wave.vactoair(mage_ThAr_vac['wave']*units.AA)

In [12]:
cut = (mage_ThAr_air > 3100*units.AA) & (mage_ThAr_air < 3400*units.AA)
mage_ThAr_air[cut]

<Quantity [3139.16171326, 3169.6408362 , 3180.2298779 , 3188.24801526,
           3195.66160954, 3232.30495299, 3238.2124784 , 3244.0536485 ,
           3247.54921961, 3253.85433904, 3257.3611663 , 3271.98840282,
           3285.81304765, 3293.62930541, 3301.64619543, 3304.24008808,
           3307.21417142, 3309.42538438, 3325.02482796, 3330.46823241,
           3333.12562665, 3348.7731695 , 3351.02749248, 3358.60059985,
           3361.18201794, 3365.26796299, 3373.33821643, 3375.55419959,
           3376.42941644, 3380.85661332, 3388.71067349, 3392.02850599,
           3397.93922477] Angstrom>

## Load up NIST + Murphy

# Generate Line List

In [14]:
create_ThArlinelist(6000.)

[INFO]    :: wavemodel.py 462 optical_modelThAr() - Creating wavelength vector in velocity space.
[INFO]    :: wavemodel.py 470 optical_modelThAr() - Add in ThAr lines
[INFO]    :: wavemodel.py 247 thar_lines() - Reading in the ThAr spectrum
[INFO]    :: wavemodel.py 574 conv2res() - The binning of the wavelength array at 6012.997927538744 is: 0.03509988663063268
[INFO]    :: wavemodel.py 578 conv2res() - Covolving with a Gaussian kernel with sigma = 12.123923254279982 pixels
[WARNING] :: wavemodel.py 493 optical_modelThAr() - Model of the ThAr spectrum outside the template coverage.
[WARNING] :: wavemodel.py 496 optical_modelThAr() - Model of the ThAr spectrum outside the template coverage.
[WARNING] :: wavemodel.py 844 create_ThArlinelist() - No min FWHM for the line detection set. Derived from the resolution at the center of the spectrum
[WARNING] :: wavemodel.py 860 create_ThArlinelist() - No file_root_name as been set. The file will called ThAr_lines.dat
[INFO]    :: wavemodel.py 

In [15]:
new_lines = waveio.load_line_list('ThAr_lines.dat')

In [16]:
new_lines[0:5]

wave,ion,NIST,Instr,amplitude,Source
float64,str4,int64,int64,float64,str12
3094.261313368305,ThAr,1,32,218.70677052723238,wavemodel.py
3140.0171522371547,ThAr,1,32,148.12281124090538,wavemodel.py
3170.5540268969266,ThAr,1,32,158.8138285062605,wavemodel.py
3181.1243070684113,ThAr,1,32,227.88057703827917,wavemodel.py
3189.181869102448,ThAr,1,32,222.56046465150408,wavemodel.py


In [17]:
new_ThAr_air = wave.vactoair(new_lines['wave']*units.AA)

In [18]:
cut = (new_ThAr_air > 3100*units.AA) & (new_ThAr_air < 3400*units.AA)
new_ThAr_air[cut]

<Quantity [3139.1076313 , 3169.63680949, 3180.20442186, 3188.25994902,
           3232.28946636, 3238.12379415, 3243.72529216, 3244.25971499,
           3247.565541  , 3253.85088608, 3257.38308597, 3271.99762246,
           3285.77501871, 3293.67535863, 3301.65007668, 3304.23829914,
           3309.38701398, 3325.01150979, 3330.4666681 , 3333.12763017,
           3348.77740657, 3351.02952968, 3361.15576552, 3374.97437086,
           3376.44469038, 3380.85697774, 3388.52667378, 3392.01290371,
           3397.57771627, 3398.51353782] Angstrom>

In [27]:
def xidl_hires(xidl_file, specbin=1):
    """
    Read an XIDL format solution for Keck/HIRES
    Note:  They used air
    Args:
        xidl_file (str):
            Keck/HIRES save file
    Returns:
    """
    xidl_dict = readsav(xidl_file)
    order_vec = xidl_dict['guess_ordr']
    norders = order_vec.size
    nspec = xidl_dict['sv_aspec'].shape[1]

    # Wavelengths
    wave = np.zeros((norders, specbin*nspec))
    spec = np.zeros((norders, specbin*nspec))

    calib = xidl_dict['all_arcfit']
    order_mask = np.ones(norders, dtype=bool)

    # Here we go on the fits
    for kk in range(norders):
        # Generate the wavelengths
        if calib['FUNC'][kk] == b'CHEBY':
            log10_wv_air = cheby_val(calib['FFIT'][kk], 
                               np.arange(nspec),
                        calib['NRM'][kk], calib['NORD'][kk])
        elif calib['FUNC'][kk] == b'POLY':
            log10_wv_air = templates.poly_val(calib['FFIT'][kk], 
                              np.arange(nspec),
                              calib['NRM'][kk])
        else:
            order_mask[kk]=False
            continue

        wv_vac = airtovac(10**log10_wv_air * units.AA).value
        ispec = xidl_dict['sv_aspec'][kk,:]
        # Flip to blue to red?
        if wv_vac[1] < wv_vac[0]:
            wv_vac = wv_vac[::-1]
            ispec = ispec[::-1]
        # Fill
        if specbin != 1:
            wave[kk,:] = arc.resize_spec(wv_vac, nspec*specbin)
            spec[kk,:] = arc.resize_spec(ispec, nspec*specbin)
        else:
            wave[kk,:] = wv_vac
            spec[kk,:] = ispec
    # Return

    return order_vec[order_mask], wave[order_mask,:], spec[order_mask,:]

In [28]:
xidl_file = '/home/xavier/Projects/xidl/Keck/HIRES/CALIBS/ARCS/hires_tmpl2x1B0.idl'

In [29]:
orders, wave, spec = xidl_hires(xidl_file)

In [30]:
orders

array([ 89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
       102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113],
      dtype=int32)

In [31]:
wave.shape

(25, 3990)

----

# Load up

In [8]:
path = '/scratch/REDUX/VLT/XShooter/UVB/Wavecal'

## merge2d

In [4]:
merge2d_file = os.path.join(path, 'CAL_SLIT_MERGE2D_UVB.fits')

In [5]:
merge2d = fits.open(merge2d_file)
merge2d.info()

Filename: /scratch/REDUX/VLT/XShooter/UVB/Wavecal/CAL_SLIT_MERGE2D_UVB.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  FLUX          1 PrimaryHDU     485   (10283, 73)   float32   
  1  ERRS          1 ImageHDU        31   (10283, 73)   float32   
  2  QUAL          1 ImageHDU        30   (10283, 73)   int32   


In [6]:
head0_merge2d = merge2d[0].header

In [7]:
head0_merge2d

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                  -32 / number of bits per data pixel                  
NAXIS   =                    2 / number of data axes                            
NAXIS1  =                10283 / length of data axis 1                          
NAXIS2  =                   73 / length of data axis 2                          
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
DATE    = '2020-05-07T11:48:58' / file creation date (YYYY-MM-DDThh:mm:ss UT)   
CRPIX1  =                   1.                                                  
CRVAL1  =              298.925                                                  
CDELT1  =                0.025                                                  
CTYPE1  = 'LINEAR  '        

## coadd2d

In [11]:
coadd2d_file = os.path.join(path, 'CAL_SLIT_ORDER2D_UVB.fits')

In [12]:
coadd2d = fits.open(coadd2d_file)
coadd2d.info()

Filename: /scratch/REDUX/VLT/XShooter/UVB/Wavecal/CAL_SLIT_ORDER2D_UVB.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  ORD13_FLUX    1 PrimaryHDU     487   (454, 73)   float32   
  1  ORD13_ERRS    1 ImageHDU        33   (454, 73)   float32   
  2  ORD13_QUAL    1 ImageHDU        33   (454, 73)   float32   
  3  ORD14_FLUX    1 ImageHDU       348   (1780, 73)   float32   
  4  ORD14_ERRS    1 ImageHDU        34   (1780, 73)   float32   
  5  ORD14_QUAL    1 ImageHDU        34   (1780, 73)   float32   
  6  ORD15_FLUX    1 ImageHDU       348   (1667, 73)   float32   
  7  ORD15_ERRS    1 ImageHDU        34   (1667, 73)   float32   
  8  ORD15_QUAL    1 ImageHDU        34   (1667, 73)   float32   
  9  ORD16_FLUX    1 ImageHDU       348   (1562, 73)   float32   
 10  ORD16_ERRS    1 ImageHDU        34   (1562, 73)   float32   
 11  ORD16_QUAL    1 ImageHDU        34   (1562, 73)   float32   
 12  ORD17_FLUX    1 ImageHDU       348   (1447, 73)   float32   
 13  OR